# Introduction
We need to properly include lipid metabolism in the model. For this we will use the restrictive approach, where we need to ensure that we can produce the three lipid heads mentioned in beata's thesis. Then we need to generate a 'general'-acylcoa molecule that reflects the distribution of fatty acids found in te Tang et al. 2009 paper. We can then combine these two into the three 'average' glycerolipids and then include these into the biomass reaction.

For the branch chain fatty acids, we need to find some data that can specify in our strain what distribution of branches fatty acids we have. For now, in this notebook I will focus on first getting the phopholipid heads in place, then make sure the production of our 'general' acylcoa is correct and combine them together into our three lipids that can be added into biomass.

In [1]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

## Glycerolipid heads
Here, from Beata's thesis we have data on the three main types of lipids present in our strain: phosphatidylglycerol, diphosphatidylglycerol and phosphatidyl ethanolamine. Here I will check that all three of these can be synthesized properly.

Note: Here I will make sure the metabolites and pathways are present. Probably, at a later time point when the generic acyl-tail has been defined we will need to revist the chemical formulas of the metabolites here to capture the different lipid types.

__Phosphatidyl glycerol__
can be made in a two step process from CDP-diacylglycerol which is present in out model as cdpdag_c.

In [4]:
#first add the phosphatidylglycerol metabolite
model.add_metabolites(Metabolite(id='pg_c', name = 'Phosphatidylglycerol', compartment = 'c', formula = 'C8H13O10PR2', charge = 0))

In [5]:
model.metabolites.pg_c.annotation['kegg.compound'] = 'C00344'
model.metabolites.pg_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pg_c.annotation['chebi'] = 'CHEBI:17517'

In [6]:
#add phosphatidylglycerophosphate metabolite
model.add_metabolites(Metabolite(id='pgp_c', name = 'Phosphatidylglycerophosphate', compartment = 'c', formula = 'C8H14O13P2R2', charge = 0))

In [7]:
model.metabolites.pgp_c.annotation['kegg.compound'] = 'C03892'
model.metabolites.pgp_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pgp_c.annotation['chebi'] = 'CHEBI:37393'

In [8]:
#add reaction from pgp to pg
model.add_reaction(Reaction(id='PGP', name = 'Phosphatidylglycerophosphate phosphohydrolase'))

In [9]:
model.reactions.PGP.annotation['kegg.reaction'] = 'R02029'
model.reactions.PGP.annotation['sbo'] = 'SBO:0000176'
model.reactions.PGP.annotation['ec-code'] = '3.1.3.27'

In [10]:
model.reactions.PGP.add_metabolites({
    model.metabolites.pgp_c:-1,
    model.metabolites.h2o_c:-1,
    model.metabolites.pg_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.h_c: 2
})

In [11]:
#add reaction converting CDP-diacyl glycerol to pgp_c
model.add_reaction(Reaction(id='CDPPT', name = 'glycerophosphate phosphatidyltransferase;'))

In [12]:
model.reactions.CDPPT.annotation['kegg.reaction'] = 'R01801'
model.reactions.CDPPT.annotation['sbo'] = 'SBO:0000176'
model.reactions.CDPPT.annotation['ec-code'] = '2.7.8.5'

In [13]:
model.reactions.CDPPT.add_metabolites({
    model.metabolites.cdpdag_c:-1,
    model.metabolites.glyc3p_c:-1,
    model.metabolites.cmp_c:1,
    model.metabolites.pgp_c:1,
    model.metabolites.h_c:-3
})

In [14]:
model.metabolites.pa_EC_c.id = 'pa_c'

So now, conversion of cdpdag_c to pg_c is possible. But just to be sure, I will go further back and check cdpdag_c can be synthesized. It seems that this is linked to central metabolism and so I will leave it for now. 

In [16]:
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

__phosphatidyl ethanolamine__ should be made in two steps from CDP-dag too (via phosphatidyl-serine). Phosphatidyl serine can currently be made from CDPDAGPT reaction, from serine and cdpdag, which is correct. We just need to add the conversion from ps to phosphatidyl-ethanolamine. We also need to add the phosphoethanolamine metabolite . 

In [33]:
model.metabolites.ps_cho_c.id = 'ps_c'

In [34]:
#add phosphoethanolamine metabolite
model.add_metabolites(Metabolite(id='pe_c', name = 'Phosphatidylethanolamine', compartment = 'c', formula = 'C7H12NO8PR2', charge = 0))

In [35]:
model.metabolites.pe_c.annotation['kegg.compound'] = 'C00350'
model.metabolites.pe_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.pe_c.annotation['chebi'] = 'CHEBI:16038'

In [36]:
#add production of phosphoethanolamine
model.add_reaction(Reaction(id='PSD', name = 'phosphatidylserine decarboxylase'))

In [37]:
model.reactions.PSD.annotation['kegg.reaction'] = 'R02055'
model.reactions.PSD.annotation['sbo'] = 'SBO:0000176'
model.reactions.PSD.annotation['ec-code'] = '4.1.1.65'

In [41]:
model.reactions.PSD.add_metabolites({
    model.metabolites.ps_c:-1,
    model.metabolites.pe_c:1,
    model.metabolites.co2_c:1
})

In [43]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__Diphosphatidylglycerol__ (AKA Cardiolipin) can be synthesized from phosphatidyl-glycerol. I will need to add the cardiolipin metabolite and conversion to it here.


In [44]:
#add cardiolipin metabolite
model.add_metabolites(Metabolite(id='clpn_c', name ='Cardiolipin', compartment = 'c', formula = 'C13H18O17P2R4', charge = 0))

In [45]:
model.metabolites.clpn_c.notes['NOTES'] = 'Cardiolipin; Diphosphatidylglycerol; 1,3-Bis(1,2-diacyl-sn-glycero-3-phospho)-sn-glycerol'

In [46]:
model.metabolites.clpn_c.annotation['kegg.compound'] = 'C05980'
model.metabolites.clpn_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.clpn_c.annotation['chebi'] = 'CHEBI:28494'

In [47]:
#add reaction
model.add_reaction(Reaction(id='CLPNS', name = 'Cardiolipin synthase'))

In [48]:
model.reactions.CLPNS.annotation['kegg.reaction'] = 'R07390'
model.reactions.CLPNS.annotation['sbo'] = 'SBO:0000176'
model.reactions.CLPNS.annotation['ec-code'] = '2.7.8.-'

In [49]:
model.reactions.CLPNS.add_metabolites({
    model.metabolites.pg_c:-2,
    model.metabolites.clpn_c:1,
    model.metabolites.glyc_c:1
})

In [50]:
reactions = [model.reactions.PGP, model.reactions.CDPPT, model.reactions.PSD, model.reactions.CLPNS]

In [51]:
model.groups.get_by_id('00564 - Glycerophospholipid metabolism').add_members(reactions)

In [53]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Acyl-CoA Tails
Now that the heads are in place, we need to make sure that each Acyl-CoA tail that has been detected in our data set can be made. From these different tail, we will then combine them with a specific data-defined stoichiometry into a general acyl-coa tail which will be used and combined with the different glycerolipid heads. 

So here I will go through the list of (branched) fatty acids in the supplementary data of Tang et al. 2009 and ensure their production.
All the tails we need to produce: 14:0 ISO, 14:0, 15:0 ISO,15:0 ANTEISO,15:0, 16:0 ISO, 16:0, 17:0 ISO,17:0 ANTEISO, 17:0, 18:4 w3c, 18:0           

First I will make sure that we can correctly produce Acyl-ACP and Malonyl-ACP which are needed to start the saturated unbranched fatty acid biosynthesis. 

Before I continue it is very important how we define these metabolites for mass balance. __The ACP-Chain WILL BE INDICATED AS C11H20N2O7PRS__. So all carbon chains should be added from the sulfur onwards. 

__Acyl-ACP__ Seems to be fine, except a duplicate reaction

In [54]:
#AMACT and KAS14 are duplicate reactions
model.remove_reactions(model.reactions.AMACT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



__Malonyl-ACP__ both malcoa and malACP seem to be produced fine. 

### Even carbon saturated, unbranched Fatty Acids
Here, we first go to the production of butyryl-ACP through hexanoyl-ACP, and from there every 'cycle' add a C2 moeity (Malonyl-ACP) to the growing chain, all the way up to the final nC18:0 that we need. 

In [126]:
model.metabolites.butacp_c.id = 'butACP_c'

AttributeError: DictList has no attribute or entry butacp_c

In [127]:
model.reactions.BUTACPAT.id = '3OAS60'

AttributeError: DictList has no attribute or entry BUTACPAT

In [40]:
model.reactions.BUTACPOR.id = 'EAR40x'

In [41]:
model.reactions.EAR40x.bounds = (-1000,0)

In [42]:
# Add reaction with NADPH here too
model.add_reaction(Reaction(id='EAR40y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C4:0)'))

In [43]:
model.reactions.EAR40y.annotation = model_e_coli.reactions.EAR40y.annotation

In [44]:
model.reactions.EAR40y.add_metabolites({
    model.metabolites.but2eACP_c: -1,
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.butACP_c:1,
    model.metabolites.nadp_c:1
})

In [45]:
model.reactions.get_by_id('3HAD40_1').bounds = (0,1000)

In [46]:
#add conversion from 3hbutACP to actACP

In [47]:
model.add_reaction(Reaction(id='3OAR40_1', name ='3-oxoacyl-[acyl-carrier-protein] reductase (n-C4:0)'))

In [48]:
model.reactions.get_by_id('3OAR40_1').annotation['sbo'] = 'SBO:0000176'
model.reactions.get_by_id('3OAR40_1').annotation['kegg.reaction'] = 'R02767'
model.reactions.get_by_id('3OAR40_1').annotation['ec-code'] = '1.1.1.100'

In [49]:
model.reactions.get_by_id('3OAR40_1').add_metabolites({
    model.metabolites.get_by_id('3hbutACP_c'): 1,
    model.metabolites.actACP_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.h_c:-1
})

In [50]:
model.metabolites.get_by_id('3hbutACP_c').formula = 'C15H27N2O9PRS'

In [51]:
#missing KAS 15 reaction
model.add_reaction(Reaction(id='KAS15', name = 'Beta-ketoacyl-ACP synthase (2)'))

In [52]:
model.reactions.KAS15.annotation = model_e_coli.reactions.KAS15.annotation

In [53]:
model.reactions.KAS15.add_metabolites({
    model.metabolites.accoa_c:-1,
    model.metabolites.h_c:-1,
    model.metabolites.malACP_c:-1,
    model.metabolites.actACP_c:1,
    model.metabolites.co2_c:1, 
    model.metabolites.coa_c:1
})

In [54]:
#Synthesis up to butyryl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

now move to C6:0 (hexanoyl-ACP)

In [55]:
#wrong ID
model.reactions.HACPOR.id = 'EAR60x'

In [56]:
model.reactions.EAR60x.bounds = (-1000,0)

In [57]:
#finished up to hexanoyl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

now onto octanoyl-ACP

In [58]:
model.reactions.HEXACPAT.id = '3OAS80'

In [59]:
model.reactions.get_by_id('3HAD80').bounds = (0,1000)

In [60]:
model.reactions.OCTACPOR.id = 'EAR80x'

In [61]:
model.reactions.EAR80x.bounds = (-1000,0)

In [62]:
model.metabolites.octacp_c.id = 'ocACP_c'

In [63]:
# add EAR80y reaction
model.add_reaction(Reaction(id='EAR80y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C8:0)'))

In [64]:
model.reactions.EAR80y.annotation = model_e_coli.reactions.EAR80y.annotation

In [65]:
model.reactions.EAR80y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.toct2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ocACP_c:1
})

In [66]:
#finished up to octanoyl-ACP
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Now on to C10:0.

In [67]:
model.reactions.OCTACPAT.id = '3OAS100'

In [68]:
model.reactions.get_by_id('3HAD100').bounds = (0,1000)

In [69]:
model.reactions.DECACPOR.id = 'EAR100x'

In [70]:
model.reactions.EAR100x.bounds = (-1000,0)

In [71]:
#add EAR100y
model.add_reaction(Reaction(id='EAR100y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C10:0)'))

In [72]:
model.reactions.EAR100y.annotation = model_e_coli.reactions.EAR100y.annotation

In [73]:
model.reactions.EAR100y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tdec2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.decanoylacp_c:1
})

In [74]:
model.metabolites.decanoylacp_c.id = 'dcaACP_c'

In [75]:
model.reactions.EAR100y.bounds = (0,1000)

In [76]:
#finished up to C10:0
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Onto C12:0

In [77]:
model.reactions.DECACPAT.id = '3OAS120'

In [78]:
model.reactions.get_by_id('3OAS120').bounds = (0,1000)

In [79]:
model.reactions.DDECACPOR.id = 'EAR120x'

In [80]:
model.reactions.EAR120x.bounds = (-1000,0)

In [81]:
#add EAR120y reaction
model.add_reaction(Reaction(id='EAR120y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C12:0)'))

In [82]:
model.reactions.EAR120y.annotation = model_e_coli.reactions.EAR120y.annotation

In [83]:
model.metabolites.dodecacp_c.id = 'ddcaACP_c'

In [84]:
model.reactions.EAR120y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tddec2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ddcaACP_c :1
})

In [85]:
model.reactions.EAR120y.bounds = (0,1000)

In [86]:
#finished up to C12:0
#save& commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

Now onto C14:0

In [87]:
model.reactions.MALACPAT.id = '3OAS140'

In [88]:
model.reactions.get_by_id('3HAD140').bounds = (0,1000)

In [89]:
model.reactions.MYRACPOR.id = 'EAR140x'

In [90]:
model.reactions.EAR140x.bounds = (-1000,0)

In [91]:
#add EAR140y reaction
model.add_reaction(Reaction(id='EAR140y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C14:0)'))

In [92]:
model.reactions.EAR140y.annotation = model_e_coli.reactions.EAR140y.annotation

In [93]:
model.reactions.EAR140y.bounds = (0,1000)

In [94]:
model.reactions.EAR140y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tmrs2eACP_c:-1,
    model.metabolites.myrsACP_c:1,
    model.metabolites.nadp_c:1
})

In [95]:
model.metabolites.myrsACP_c.name = 'Tetradecanoyl-[acp]'

In [96]:
#finished up to C14:0, 
#save&commit
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

Now onto C16:0

In [97]:
model.reactions.MMACPAT.id = '3OAS160'

In [98]:
model.metabolites.get_by_id('3oxhdacp_c').id = '3opalmACP_c'

In [99]:
model.reactions.get_by_id('3HAD160').bounds = (0,1000)

In [100]:
model.reactions.OPALMACPOR.id = 'EAR160x'

In [101]:
model.reactions.EAR160x.bounds = (-1000,0)

In [102]:
model.metabolites.hdeacp_c.id = 'palmACP_c'

In [103]:
#add EAR160y
model.add_reaction(Reaction(id='EAR160y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C16:0)'))

In [104]:
model.reactions.EAR160y.annotation = model_e_coli.reactions.EAR160y.annotation

In [105]:
model.reactions.EAR160y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.tpalm2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.palmACP_c:1
})

In [106]:
#finished until c16:0
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

Now onto C18:0

In [107]:
model.reactions.HDEACPT.id = '3OAS180'

In [108]:
model.reactions.OXSTACPOR.id = '3OAR180'

In [109]:
model.reactions.get_by_id('3HAD180').bounds = (0,1000)

In [110]:
model.reactions.TOCTDACPOR.id = 'EAR180x'

In [111]:
model.reactions.EAR180x.bounds = (0,1000)

In [112]:
#add EAR180y
model.add_reaction(Reaction(id='EAR180y', name = 'Enoyl-[acyl-carrier-protein] reductase (NADPH) (n-C18:0)'))

In [113]:
model.reactions.EAR180y.annotation = model_e_coli.reactions.EAR180y.annotation

In [114]:
model.reactions.EAR180y.add_metabolites({
    model.metabolites.h_c:-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.toctd2eACP_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.ocdcaACP_c:1
})

In [115]:
model.reactions.EAR180y.gene_reaction_rule = model.reactions.EAR180x.gene_reaction_rule

In [116]:
model.reactions.EAR160y.gene_reaction_rule = model.reactions.EAR160x.gene_reaction_rule
model.reactions.EAR140y.gene_reaction_rule = model.reactions.EAR140x.gene_reaction_rule
model.reactions.EAR120y.gene_reaction_rule = model.reactions.EAR120x.gene_reaction_rule
model.reactions.EAR100y.gene_reaction_rule = model.reactions.EAR100x.gene_reaction_rule
model.reactions.EAR80y.gene_reaction_rule = model.reactions.EAR80x.gene_reaction_rule
model.reactions.EAR60y.gene_reaction_rule = model.reactions.EAR60x.gene_reaction_rule
model.reactions.EAR40y.gene_reaction_rule = model.reactions.EAR40x.gene_reaction_rule

In [117]:
#finished up to C18:0
cobra.io.write_sbml_model(model,'../../model/g-thermo.xml')

Now I've finished making sure the pathway for the generation of the saturated, unbranched fatty acids is correct. Next I will tackle the unbranched, uneven carbon length fatty acids. 

### Unbranched, saturated odd-carbon fatty acids

These are made in the same way as the Even-chain saturated FAs, but the first molecule of the process is propionyl-CoA (C3) instead of acetyl-CoA (C2). So here I will go all the way from the C3 to the C17:0 pathway. 
So here, I will add all the steps in the pathway similarly to the even chain fatty acids.

In [118]:
model = cobra.io.read_sbml_model('../../model/g-thermo.xml')

In [128]:
#first need to form propionyl-ACP from prorionyl-Coa, similar to the MCOATA reaction
#first add propionyl-ACP met
model.add_metabolites(Metabolite(id='propACP_c', name = 'Propionyl-ACP', charge =-1, compartment = 'c', formula = 'C14H25N2O8PRS'))

In [129]:
model.metabolites.propACP_c.annotation['sbo'] = 'SBO:0000247'

In [130]:
#add reaction
model.add_reaction(Reaction(id='PCOATA', name = 'Propanyl-CoA:[acyl-carrier-protein] S-malonyltransferase'))

Ignoring reaction 'PCOATA' since it already exists.


In [131]:
model.reactions.PCOATA.annotation['sbo'] = 'SBO:0000176'

In [132]:
model.reactions.PCOATA.add_metabolites({
    model.metabolites.ppcoa_c:-1,
    model.metabolites.ACP_c:-1,
    model.metabolites.propACP_c:1,
    model.metabolites.coa_c:1,
    model.metabolites.h_c:1
})

In [133]:
model.reactions.PCOATA.bounds = (-1000,1000)

In [134]:
#add condenstation of prop-ACP with malonyl ACP to 3-oxopentanoyl-ACP
#add metabolite
model.add_metabolites(Metabolite(id='3opentACP_c', name = '3-oxopentanoyl-ACP', compartment = 'c', charge =-1, formula = 'C16H27O9N2PRS'))

In [135]:
model.metabolites.get_by_id('3opentACP_c').annotation['sbo'] = 'SBO:0000176'

In [136]:
#add reaction
model.add_reaction(Reaction(id='3OAS50', name = '3-oxopentanoyl-[acyl-carrier-protein] synthase (n-C5:0)'))

In [137]:
model.reactions.get_by_id('3OAS50').annotation['sbo'] = 'SBO:0000176'

In [138]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAS50').gene_reaction_rule = 'RTMO12345'

In [139]:
model.reactions.get_by_id('3OAS50').add_metabolites({
    model.metabolites.malACP_c:-1,
    model.metabolites.propACP_c:-1,
    model.metabolites.ACP_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.get_by_id('3opentACP_c'):1,
    model.metabolites.h_c:-1
})

In [148]:
#next add reaction with NADPH
#first add 3-Hydroxypentanoyl-[acp] met
model.add_metabolites(Metabolite(id='3hpentACP_c', name = '3-Hydroxypentanoyl-[acp]', compartment = 'c', charge = -1, formula = 'C16H29O9N2PRS'))

In [149]:
model.metabolites.get_by_id('3hpentACP_c').annotation['sbo'] = 'SBO:0000247'

In [158]:
#add reaction
model.add_reaction(Reaction(id='3OAR50', name = ' 3-oxoacyl-[acyl-carrier-protein] reductase (n-C5:0)'))

In [159]:
model.reactions.get_by_id('3OAR50').annotation['sbo'] = 'SBO:0000176'

In [160]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3OAR50').gene_reaction_rule = 'RTMO12345'

In [161]:
model.reactions.get_by_id('3OAR50').add_metabolites({
    model.metabolites.get_by_id('3opentACP_c'):-1,
    model.metabolites.nadph_c:-1,
    model.metabolites.nadp_c:1,
    model.metabolites.get_by_id('3hpentACP_c'):1,
    model.metabolites.h_c:-1
})

In [165]:
#next add the tpent2eACP_c metabolite
model.add_metabolites(Metabolite(id='tpent2eACP_c', name = 'Trans-Pent-2-enoyl-[acyl-carrier protein]', compartment = 'c', charge = -1, formula = 'C16H27O8N2PRS'))

In [166]:
model.metabolites.get_by_id('tpent2eACP_c').annotation['sbo'] = 'SBO:0000247'

In [169]:
#add reaction
model.add_reaction(Reaction(id='3HAD50', name = '3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C5:0)'))

In [170]:
model.reactions.get_by_id('3HAD50').annotation['sbo'] = 'SBO:0000176'

In [171]:
#assume GPR is the same FAget_by_id('3OAS50')synthase: RTMO12345
model.reactions.get_by_id('3HAD50').gene_reaction_rule = 'RTMO12345'

In [172]:
model.reactions.get_by_id('3HAD50').add_metabolites({
    model.metabolites.get_by_id('3hpentACP_c'):-1,
    model.metabolites.h2o_c:1,
    model.metabolites.tpent2eACP_c:1
})

In [ ]:
#last step here is thhe conversion to C7:0
#ass both NADPH and NADH reactions or just one? 

Once I've finished this one cycle, save & commit
then I can copy this one complete cycle and just clowly go through it and fix it every time. 
As the unbrnahc chain synthesis is not in kegg, i will not include it in the group.

In [163]:
model.metabolites.get_by_id('3hpentACP_c')

Metabolite identifier,3hpentACP_c
Name,3-Hydroxypentanoyl-[acp]
Memory address,0x0220df4dc308
Formula,C16H29O9N2PRS
Compartment,c
In 1 reaction(s),3OAR50


For each cycle: add 4 reactions:
1) 3OASXX: Addition of amalonyl-ACP to the growing ACP-chain. Adding a C2OH2 to the chain to give the 3oXXACP_c metabolite

2) 3OARXX: reducing double bond with NADPH addition (REVERSIBLE). Just adds H2 to the metabolite to give 3hXXXACP_c metabolite
    - name: 3-oxoacyl-[acyl-carrier-protein] reductase (n-CX:0)
    - charge: no change

3) 3HADXX: removed H2O (irreversible) from the met to give tXX2eACP_c metabolite
    - name : 3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-CX:0)


4) EARXXx and EARXXy: met reacts with NAD(P)H to give the Cn+2-ACP metabolite that can then enter the next cycle again.

Do this all the way up until C17:0. 

https://en.wikipedia.org/wiki/List_of_saturated_fatty_acids 
https://en.wikipedia.org/wiki/Fatty_acid_synthesis#/media/File:Saturated_Fatty_Acid_Synthesis.svg

In [167]:
model.metabolites.thex2eACP_c.summary()

In [168]:
model_e_coli.reactions.get_by_id('3HAD60')

Reaction identifier,3HAD60
Name,3-hydroxyacyl-[acyl-carrier-protein] dehydratase (n-C6:0)
Memory address,0x0220d9d40fc8
Stoichiometry,3hhexACP_c --> h2o_c + thex2eACP_c (R)-3-Hydroxyhexanoyl-[acyl-carrier protein] --> H2O H2O + Trans-Hex-2-enoyl-[acyl-carrier protein]
GPR,b0954 or b0180
Lower bound,0.0
Upper bound,1000.0


In [326]:
model.metabolites.propACP_c

Metabolite identifier,propACP_c
Name,Propionyl-ACP
Memory address,0x0260a1466448
Formula,C14H25N2O8PRS
Compartment,c
In 2 reaction(s),"3OAS50, PCOATA"


In [157]:
model.reactions.get_by_id('3OAR60')

Reaction identifier,3OAR60
Name,(3R)-3-Hydroxyhexanoyl-[acyl-carrier-protein]:NADP+ oxidoreductase
Memory address,0x0220df8821c8
Stoichiometry,3hhexACP_c + nadp_c <=> 3ohexACP_c + h_c + nadph_c 3-Hydroxyhexanoyl-[acp] + NADP+ <=> 3-Oxohexanoyl-[acp] + H+ + NADPH
GPR,RTMO12345
Lower bound,-1000.0
Upper bound,1000.0


In [245]:
model.metabolites.ppcoa_c.summary()

In [288]:
model_e_coli.metabolites.malACP_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,MCOATA,100.00000,0.068255,ACP_c + malcoa_c <=> coa_c + malACP_c
CONSUMING,3OAS140,99.99486,0.068251,ddcaACP_c + h_c + malACP_c --> 3omrsACP_c + AC...


In [253]:
model.reactions.EAR80x

Reaction identifier,EAR80x
Name,octanoyl-[acp]:NAD+ trans-2-oxidoreductase
Memory address,0x0260a8cb4e88
Stoichiometry,nad_c + ocACP_c <-- h_c + nadh_c + toct2eACP_c NAD+ + Octanoyl-[acp] <-- H+ + NADH + trans-Oct-2-enoyl-[acp]
GPR,RTMO12345
Lower bound,-1000
Upper bound,0


In [ ]:
RTMO12345

In [ ]:
#add all new reactions to the fatty acid biosynthesis group
reactions = [model.reactions.EAR160y, model.reactions.EAR180y,model.reactions.EAR140y, model.reactions.EAR120y, model.reactions.EAR40y,model.reactions.EAR80y, model.reactions.get_by_id('3OAR40_1'), model.reactions.KAS15, model.reactions.EAR100y]

In [ ]:
#check all mass balances later again!! as this ACP stuff is an issue.

Go through the pahway for C14, C16 and C18. Then check mass balance is still fine. Then remove them from the 'todo' list above. Save & commit and push so ben can take a look.
Wait what is said in the question i raised about the EAR reactions



then work on the un-even saturated unbranched chains. 

15:0 ISO
15:0 ANTEISO
15:0
16:0 ISO
17:0 ISO
17:0 ANTEISO
17:0
18:4 w3c

Done:
- 14:0 : met myrsACP_c
- 16:0: met palACP_c
- 18:0: met ocdcaACP_c